In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

### Passo a Passo

O objetivo é criar blocos de instrução de uma rede neural

### Zero page

Colocar zeros envolta das imagens para:

- Ajuda na hora de transforma a imagem para não perde qualidade das bordas na hora de transforma ela em um tamanho menor, pois, a borda não tem outros pixels do lado

- Ajuda a manter todas as informações necessárias

In [4]:
def zero_pad(X, pad):

  X_pad = np.pad(X, ((0,0), (pad,pad), (pad,pad), (0,0)), mode='constant', contant_values= (0,0))

### Passo unico de Convolução

Cria um filtro para realizar um passo do convolução:

- Onde pega o volume da entrada
- Aplica o Filtro
- Sai um outro volume de entrada


Nessa primeira etapa vamos utilizar apenas um filtro e aplicar em uma saida

In [ ]:
def conv_single_step(a_slice_prev, W, b):

  s = np.multiply(a_slice_prev, W)
  Z = np.sum(s)
  Z = Z +float(b)